In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("Resources\exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
#selected_features = df.drop("koi_disposition", axis=1)

selected_features = df.drop(["koi_disposition",
                             "koi_tce_plnt_num"
                            ],axis=1)

selected_features.head(25)

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
5,0,0,0,0,2.566589,1.780000e-05,-1.780000e-05,179.554370,0.004610,-0.004610,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,0,0,0,0,16.068647,1.090000e-05,-1.090000e-05,173.621937,0.000517,-0.000517,...,-83,4.485,0.083,-0.028,0.848,0.033,-0.072,286.99948,48.375790,15.841
7,0,0,0,0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,...,-78,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,49.316399,11.338
8,0,1,0,0,2.204735,4.300000e-08,-4.300000e-08,121.358542,0.000016,-0.000016,...,-89,4.019,0.033,-0.027,1.952,0.099,-0.110,292.24728,47.969521,10.463
9,0,0,0,0,3.522498,1.980000e-07,-1.980000e-07,121.119423,0.000047,-0.000047,...,-137,4.169,0.055,-0.045,1.451,0.110,-0.110,281.28812,42.451080,13.563


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
X = selected_features
y = df["koi_disposition"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, train_size=0.65,test_size=0.35)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4430,1,0,0,0,149.237142,0.002931,-0.002931,168.15630,0.01640,-0.01640,...,-142,4.576,0.027,-0.202,0.812,0.249,-0.062,288.32599,41.027618,15.683
2330,0,0,0,0,20.605501,0.000259,-0.000259,146.50690,0.01020,-0.01020,...,-85,4.193,0.016,-0.018,1.308,0.054,-0.054,285.47659,41.632717,11.049
4356,0,0,0,0,3.290304,0.000004,-0.000004,131.90944,0.00178,-0.00178,...,-33,4.810,0.010,-0.020,0.427,0.036,-0.009,292.57986,37.376411,15.669
2142,0,0,0,0,8.073941,0.000048,-0.000048,540.04161,0.00487,-0.00487,...,-82,4.869,0.049,-0.055,0.398,0.049,-0.049,285.73981,40.980049,16.109
856,0,0,0,0,21.060753,0.000057,-0.000057,143.59983,0.00227,-0.00227,...,-103,4.382,0.099,-0.110,1.034,0.148,-0.121,298.17722,42.237041,13.182


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [9]:
# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=500)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

0.8912954638332652

In [10]:
# predecictions
predictions = rf.predict(X_test_scaled)
print(predictions)

['FALSE POSITIVE' 'FALSE POSITIVE' 'CONFIRMED' ... 'FALSE POSITIVE'
 'CONFIRMED' 'CANDIDATE']


In [11]:
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8912954638332652


In [12]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.09208761, 0.06887812, 0.10988141, 0.03582401, 0.02341652,
       0.01896324, 0.01793715, 0.0137784 , 0.02347996, 0.02191575,
       0.01857711, 0.01094359, 0.01081737, 0.02544477, 0.0306424 ,
       0.03326429, 0.01953966, 0.01505625, 0.01492791, 0.04873768,
       0.03506366, 0.02793749, 0.01651684, 0.01483349, 0.0173951 ,
       0.01248312, 0.05817506, 0.00973273, 0.03018895, 0.02795809,
       0.00958408, 0.00875802, 0.01069132, 0.0093149 , 0.01208082,
       0.00900058, 0.0133431 , 0.01124014, 0.01158934])

In [13]:
feature_names = selected_features.columns

sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.10988140920245197, 'koi_fpflag_co'),
 (0.09208760758808064, 'koi_fpflag_nt'),
 (0.0688781151549237, 'koi_fpflag_ss'),
 (0.05817505599071245, 'koi_model_snr'),
 (0.04873768464535853, 'koi_prad'),
 (0.0358240091516128, 'koi_fpflag_ec'),
 (0.03506366431159504, 'koi_prad_err1'),
 (0.03326428636187318, 'koi_duration_err2'),
 (0.030642404629630745, 'koi_duration_err1'),
 (0.03018895112077548, 'koi_steff_err1'),
 (0.027958085356998463, 'koi_steff_err2'),
 (0.02793748645193543, 'koi_prad_err2'),
 (0.025444770025442456, 'koi_duration'),
 (0.02347995748831168, 'koi_time0bk_err1'),
 (0.023416515970337254, 'koi_period'),
 (0.021915750737540936, 'koi_time0bk_err2'),
 (0.01953966334211579, 'koi_depth'),
 (0.018963238210776542, 'koi_period_err1'),
 (0.018577107777703418, 'koi_impact'),
 (0.01793714768412286, 'koi_period_err2'),
 (0.017395099776070493, 'koi_insol_err1'),
 (0.016516839698414547, 'koi_teq'),
 (0.015056251489941367, 'koi_depth_err1'),
 (0.014927914277970427, 'koi_depth_err2'),
 (0.01

In [14]:
print(classification_report(y_test, 
                            predictions, 
                            target_names = ["Confirmed", "False Positive", "Candidate"]))

                precision    recall  f1-score   support

     Confirmed       0.81      0.74      0.77       591
False Positive       0.80      0.82      0.81       630
     Candidate       0.97      1.00      0.99      1226

      accuracy                           0.89      2447
     macro avg       0.86      0.85      0.86      2447
  weighted avg       0.89      0.89      0.89      2447



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [15]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [750, 1000, 1250],
              'max_depth': [100, 200, 300],
              'criterion': ['gini', 'entropy']}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [16]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] criterion=gini, max_depth=100, n_estimators=750 .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=100, n_estimators=750, score=0.877, total=   7.8s
[CV] criterion=gini, max_depth=100, n_estimators=750 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s remaining:    0.0s


[CV]  criterion=gini, max_depth=100, n_estimators=750, score=0.895, total=   7.8s
[CV] criterion=gini, max_depth=100, n_estimators=750 .................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.6s remaining:    0.0s


[CV]  criterion=gini, max_depth=100, n_estimators=750, score=0.892, total=   7.9s
[CV] criterion=gini, max_depth=100, n_estimators=750 .................
[CV]  criterion=gini, max_depth=100, n_estimators=750, score=0.901, total=   8.0s
[CV] criterion=gini, max_depth=100, n_estimators=750 .................
[CV]  criterion=gini, max_depth=100, n_estimators=750, score=0.891, total=   8.5s
[CV] criterion=gini, max_depth=100, n_estimators=1000 ................
[CV]  criterion=gini, max_depth=100, n_estimators=1000, score=0.879, total=  11.4s
[CV] criterion=gini, max_depth=100, n_estimators=1000 ................
[CV]  criterion=gini, max_depth=100, n_estimators=1000, score=0.888, total=  11.4s
[CV] criterion=gini, max_depth=100, n_estimators=1000 ................
[CV]  criterion=gini, max_depth=100, n_estimators=1000, score=0.891, total=  11.4s
[CV] criterion=gini, max_depth=100, n_estimators=1000 ................
[CV]  criterion=gini, max_depth=100, n_estimators=1000, score=0.897, total=  11

[CV]  criterion=entropy, max_depth=100, n_estimators=1250, score=0.897, total=  22.2s
[CV] criterion=entropy, max_depth=100, n_estimators=1250 .............
[CV]  criterion=entropy, max_depth=100, n_estimators=1250, score=0.892, total=  21.8s
[CV] criterion=entropy, max_depth=100, n_estimators=1250 .............
[CV]  criterion=entropy, max_depth=100, n_estimators=1250, score=0.900, total=  23.0s
[CV] criterion=entropy, max_depth=100, n_estimators=1250 .............
[CV]  criterion=entropy, max_depth=100, n_estimators=1250, score=0.894, total=  22.0s
[CV] criterion=entropy, max_depth=200, n_estimators=750 ..............
[CV]  criterion=entropy, max_depth=200, n_estimators=750, score=0.879, total=  13.0s
[CV] criterion=entropy, max_depth=200, n_estimators=750 ..............
[CV]  criterion=entropy, max_depth=200, n_estimators=750, score=0.895, total=  13.0s
[CV] criterion=entropy, max_depth=200, n_estimators=750 ..............
[CV]  criterion=entropy, max_depth=200, n_estimators=750, sc

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 22.4min finished


GridSearchCV(estimator=RandomForestClassifier(n_estimators=500),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [100, 200, 300],
                         'n_estimators': [750, 1000, 1250]},
             verbose=3)

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 750}
0.8928262650053551


# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Models\RandomForest.sav'
joblib.dump(rf, filename)

['Models\\RandomForest.sav']